In [1]:
import polars as pl

# 8.1 Parsing Unix timestamps

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at `/var/log/popularity-contest`.

Here's an [explanation of how this file works](http://popcon.ubuntu.com/README).

I'm going to hope that nothing in it is sensitive :)

**IF THE FILE DOES NOT HAVE A HEADER, POLARS INFERS THE SCHEMA FROM THE FIRST LINE**
**SINCE FIRST LINE ONLY HAVE 4 FIELDS, IT GIVES AN ERROR FOR THE LINES WITH 5 FIELDS**
**I MANUALLY INSERTED A LINE WITH 5 FIELDS AND REMOVE LATER**

In [2]:
popcon = pl.read_csv(
    "./data/popularity-contest.csv",
    skip_rows=1,              
    separator=" ",            
    has_header=False
).slice(1, -1)         
popcon = popcon.rename({
    popcon.columns[0]: "atime",
    popcon.columns[1]: "ctime",
    popcon.columns[2]: "package-name",
    popcon.columns[3]: "mru-program",
    popcon.columns[4]: "tag"
})
popcon

atime,ctime,package-name,mru-program,tag
str,str,str,str,str
"""1387295797""","""1367633260""","""perl-base""","""/usr/bin/perl""",null
"""1387295796""","""1354370480""","""login""","""/bin/su""",null
"""1387295743""","""1354341275""","""libtalloc2""","""/usr/lib/x86_64-linux-gnu/libt…",null
"""1387295743""","""1387224204""","""libwbclient0""","""/usr/lib/x86_64-linux-gnu/libw…","""<RECENT-CTIME>"""
"""1387295742""","""1354341253""","""libselinux1""","""/lib/x86_64-linux-gnu/libselin…",null
…,…,…,…,…
"""0""","""0""","""libreadline-dev""","""<NOFILES>""",null
"""0""","""0""","""notify-osd-icons""","""<NOFILES>""",null
"""0""","""0""","""python-apt-common""","""<NOFILES>""",null


The colums are the access time, created time, package name, recently used program, and a tag

In [3]:
print(len(popcon)) #2897

2897


The magical part about parsing timestamps in pandas is that numpy datetimes are already stored as Unix timestamps. So all we need to do is tell pandas that these integers are actually datetimes -- it doesn't need to do any conversion at all.

We need to convert these to ints to start:

In [4]:
popcon = popcon.with_columns([
    pl.col('atime').cast(pl.Int64),
    pl.col('ctime').cast(pl.Int64)
])

Every numpy array and pandas series has a dtype -- this is usually `int64`, `float64`, or `object`. Some of the time types available are `datetime64[s]`, `datetime64[ms]`, and `datetime64[us]`. There are also `timedelta` types, similarly.

We can use the `pd.to_datetime` function to convert our integer timestamps into datetimes. This is a constant-time operation -- we're not actually changing any of the data, just how pandas thinks about it.

In [5]:
popcon = popcon.with_columns([
    pl.from_epoch(pl.col('atime').cast(pl.Int64), time_unit='s'),
    pl.from_epoch(pl.col('ctime').cast(pl.Int64), time_unit='s')
])

If we look at the dtype now, it's `datetime`.

In [6]:
popcon.schema

Schema([('atime', Datetime(time_unit='us', time_zone=None)),
        ('ctime', Datetime(time_unit='us', time_zone=None)),
        ('package-name', String),
        ('mru-program', String),
        ('tag', String)])

So now we can look at our `atime` and `ctime` as dates!

In [7]:
popcon[:5]

atime,ctime,package-name,mru-program,tag
datetime[μs],datetime[μs],str,str,str
2013-12-17 15:56:37,2013-05-04 02:07:40,"""perl-base""","""/usr/bin/perl""",null
2013-12-17 15:56:36,2012-12-01 14:01:20,"""login""","""/bin/su""",null
2013-12-17 15:55:43,2012-12-01 05:54:35,"""libtalloc2""","""/usr/lib/x86_64-linux-gnu/libt…",null
2013-12-17 15:55:43,2013-12-16 20:03:24,"""libwbclient0""","""/usr/lib/x86_64-linux-gnu/libw…","""<RECENT-CTIME>"""
2013-12-17 15:55:42,2012-12-01 05:54:13,"""libselinux1""","""/lib/x86_64-linux-gnu/libselin…",null


Now suppose we want to look at all packages that aren't libraries.

First, I want to get rid of everything with timestamp 0. Notice how we can just use a string in this comparison, even though it's actually a timestamp on the inside? That is because pandas is amazing.

In [8]:
popcon = popcon.filter(
    pl.col('atime') > pl.lit('1970-01-01').str.strptime(pl.Datetime, format='%Y-%m-%d')
)

#HERE
Now we can use pandas' magical string abilities to just look at rows where the package name doesn't contain 'lib'.

In [11]:
nonlibraries = popcon.filter(~pl.col('package-name').str.contains('lib'))

In [14]:
nonlibraries.sort(by='ctime', descending=True)[:10]

atime,ctime,package-name,mru-program,tag
datetime[μs],datetime[μs],str,str,str
2013-12-17 04:55:39,2013-12-17 04:55:42,"""ddd""","""/usr/bin/ddd""","""<RECENT-CTIME>"""
2013-12-16 20:03:20,2013-12-16 20:05:13,"""nodejs""","""/usr/bin/npm""","""<RECENT-CTIME>"""
2013-12-16 20:03:20,2013-12-16 20:05:04,"""thunderbird-locale-en""","""/usr/lib/thunderbird-addons/ex…","""<RECENT-CTIME>"""
2013-12-16 20:03:20,2013-12-16 20:05:04,"""switchboard-plug-keyboard""","""/usr/lib/plugs/pantheon/keyboa…","""<RECENT-CTIME>"""
2013-12-16 20:08:27,2013-12-16 20:05:03,"""software-center""","""/usr/sbin/update-software-cent…","""<RECENT-CTIME>"""
2013-12-16 20:03:20,2013-12-16 20:05:00,"""samba-common-bin""","""/usr/bin/net.samba3""","""<RECENT-CTIME>"""
2013-12-16 20:08:25,2013-12-16 20:04:59,"""postgresql-client-9.1""","""/usr/lib/postgresql/9.1/bin/ps…","""<RECENT-CTIME>"""
2013-12-16 20:08:23,2013-12-16 20:04:58,"""postgresql-9.1""","""/usr/lib/postgresql/9.1/bin/po…","""<RECENT-CTIME>"""
2013-12-16 20:03:20,2013-12-16 20:04:55,"""php5-dev""","""/usr/include/php5/main/snprint…","""<RECENT-CTIME>"""


Okay, cool, it says that I I installed ddd recently. And postgresql! I remember installing those things. Neat.

The whole message here is that if you have a timestamp in seconds or milliseconds or nanoseconds, then you can just "cast" it to a `'datetime64[the-right-thing]'` and pandas/numpy will take care of the rest.

<style>
    @font-face {
        font-family: "Computer Modern";
        src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
    }
    div.cell{
        width:800px;
        margin-left:16% !important;
        margin-right:auto;
    }
    h1 {
        font-family: Helvetica, serif;
    }
    h4{
        margin-top:12px;
        margin-bottom: 3px;
       }
    div.text_cell_render{
        font-family: Computer Modern, "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;
        line-height: 145%;
        font-size: 130%;
        width:800px;
        margin-left:auto;
        margin-right:auto;
    }
    .CodeMirror{
            font-family: "Source Code Pro", source-code-pro,Consolas, monospace;
    }
    .text_cell_render h5 {
        font-weight: 300;
        font-size: 22pt;
        color: #4057A1;
        font-style: italic;
        margin-bottom: .5em;
        margin-top: 0.5em;
        display: block;
    }
    
    .warning{
        color: rgb( 240, 20, 20 )
        }  